# Chapter 2: Framebuffer and 2D Rasterization

## Building the Foundation

This notebook covers:
- Framebuffer implementation
- Basic 2D drawing primitives (lines, circles)
- Polygon filling
- 2D clipping algorithms

**Key References:** Marschner & Shirley Ch. 3, 8, Gambetta Ch. 2-3

In [ ]:
---

## 1. Framebuffer - Theory

### 1.1 What is a Framebuffer?

A **framebuffer** is a 2D array of pixels that represents the image to be displayed on screen. Each pixel stores color information.

**Pixel representation:**
$$\text{Pixel}(x, y) = (R, G, B) \quad \text{or} \quad (R, G, B, A)$$

where:
- $R, G, B$ are red, green, blue color channels (typically 0-255 or 0.0-1.0)
- $A$ is alpha (transparency) channel (optional)
- $(x, y)$ are integer screen coordinates

### 1.2 Coordinate Systems

**Screen space coordinates:**
- Origin $(0, 0)$ typically at top-left or bottom-left
- $x$ increases rightward
- $y$ increases downward (or upward, depending on convention)

**Resolution:** Width $W$ × Height $H$ pixels

### 1.3 Color Representation

**RGB Color Space:**
$$\mathbf{c} = \begin{pmatrix} r \\ g \\ b \end{pmatrix}, \quad r, g, b \in [0, 1] \text{ or } [0, 255]$$

**Common operations:**
- **Color addition:** $\mathbf{c}_1 + \mathbf{c}_2 = (r_1+r_2, g_1+g_2, b_1+b_2)$
- **Scalar multiplication:** $k\mathbf{c} = (kr, kg, kb)$
- **Clamping:** $\text{clamp}(c, 0, 1) = \max(0, \min(c, 1))$

### 1.4 Alpha Blending

Combining colors with transparency:

$$\mathbf{c}_{\text{out}} = \alpha \mathbf{c}_{\text{src}} + (1-\alpha)\mathbf{c}_{\text{dst}}$$

where:
- $\alpha \in [0, 1]$ is opacity (0 = fully transparent, 1 = fully opaque)
- $\mathbf{c}_{\text{src}}$ is the source (new) color
- $\mathbf{c}_{\text{dst}}$ is the destination (existing) color

In [ ]:
# Example 1: Simple 2D Scene
fb = Framebuffer(800, 600, background_color=(0.1, 0.15, 0.2))

# Draw a house
# Walls (rectangle made of triangles)
draw_triangle_filled(fb, 250, 200, 550, 200, 550, 400, (0.8, 0.6, 0.4))
draw_triangle_filled(fb, 250, 200, 250, 400, 550, 400, (0.8, 0.6, 0.4))

# Roof
draw_triangle_filled(fb, 200, 400, 400, 500, 600, 400, (0.6, 0.3, 0.2))

# Door
draw_triangle_filled(fb, 350, 200, 450, 200, 450, 320, (0.4, 0.2, 0.1))
draw_triangle_filled(fb, 350, 200, 350, 320, 450, 320, (0.4, 0.2, 0.1))

# Windows
draw_circle_filled(fb, 300, 350, 25, (0.6, 0.8, 1.0))
draw_circle_filled(fb, 500, 350, 25, (0.6, 0.8, 1.0))

# Sun
draw_circle_filled(fb, 650, 500, 40, (1.0, 0.9, 0.3))

# Ground
for i in range(800):
    for j in range(200):
        # Gradient
        green = 0.3 + 0.2 * (j / 200)
        fb.set_pixel(i, j, (0.2, green, 0.1))

fb.display("Example 1: Simple 2D Scene")

# Example 2: Gradient Circles
fb = Framebuffer(600, 600, background_color=(0.0, 0.0, 0.0))

# Create overlapping gradient circles
circles = [
    (200, 300, 120, (1.0, 0.2, 0.2)),
    (300, 400, 120, (0.2, 1.0, 0.2)),
    (400, 300, 120, (0.2, 0.2, 1.0)),
]

for cx, cy, radius, base_color in circles:
    for r in range(radius, 0, -5):
        # Color fades from base to white as radius decreases
        t = 1.0 - (r / radius)
        color = tuple(c * (1 - t * 0.7) + t * 0.7 for c in base_color)
        draw_circle_midpoint(fb, cx, cy, r, color)

fb.display("Example 2: Gradient Circles with Blending")

# Example 3: Polygon Star
fb = Framebuffer(600, 600, background_color=(0.05, 0.05, 0.15))

# Draw a star using triangles
center_x, center_y = 300, 300
outer_radius = 200
inner_radius = 80
n_points = 5

# Calculate star vertices
vertices = []
for i in range(n_points * 2):
    angle = math.pi / 2 + 2 * math.pi * i / (n_points * 2)
    if i % 2 == 0:
        # Outer point
        radius = outer_radius
        color = (1.0, 0.8, 0.2)
    else:
        # Inner point
        radius = inner_radius
        color = (1.0, 0.95, 0.5)
    
    x = center_x + int(radius * math.cos(angle))
    y = center_y + int(radius * math.sin(angle))
    vertices.append((x, y, color))

# Draw star triangles
for i in range(len(vertices)):
    next_i = (i + 1) % len(vertices)
    x0, y0, c0 = vertices[i]
    x1, y1, c1 = vertices[next_i]
    
    # Draw triangle from center to each edge with color interpolation
    draw_triangle_interpolated(fb,
                               center_x, center_y, (1.0, 1.0, 0.8),
                               x0, y0, c0,
                               x1, y1, c1)

fb.display("Example 3: Star Polygon")

# Example 4: Animated-looking Pattern
fb = Framebuffer(800, 800, background_color=(0.0, 0.0, 0.0))

# Create spiral pattern
center_x, center_y = 400, 400
for i in range(200):
    angle = i * 0.5
    radius = i * 2
    
    x = center_x + int(radius * math.cos(angle))
    y = center_y + int(radius * math.sin(angle))
    
    # Color based on position
    r = (math.sin(i * 0.1) + 1) / 2
    g = (math.cos(i * 0.1) + 1) / 2
    b = (math.sin(i * 0.05 + 1) + 1) / 2
    
    draw_circle_filled(fb, x, y, 8, (r, g, b))

fb.display("Example 4: Spiral Pattern")

print("\n✓ Chapter 2 Complete!")
print("\nIn this chapter, you learned:")
print("  • Framebuffer implementation and pixel operations")
print("  • Line drawing (Bresenham and DDA algorithms)")
print("  • Circle drawing (Midpoint algorithm)")
print("  • Triangle rasterization with barycentric coordinates")
print("  • 2D clipping (Cohen-Sutherland algorithm)")
print("\nNext Chapter: 3D Geometry and Data Structures")

---

## 6. Practical Examples

Combining all the 2D rasterization techniques we've learned.

In [ ]:
# Cohen-Sutherland region codes
INSIDE = 0  # 0000
LEFT = 1    # 0001
RIGHT = 2   # 0010
BOTTOM = 4  # 0100
TOP = 8     # 1000

def compute_outcode(x: float, y: float, x_min: float, y_min: float, 
                    x_max: float, y_max: float) -> int:
    """Compute Cohen-Sutherland region code for a point"""
    code = INSIDE
    
    if x < x_min:
        code |= LEFT
    elif x > x_max:
        code |= RIGHT
    
    if y < y_min:
        code |= BOTTOM
    elif y > y_max:
        code |= TOP
    
    return code

def clip_line_cohen_sutherland(x0: float, y0: float, x1: float, y1: float,
                               x_min: float, y_min: float, x_max: float, y_max: float):
    """
    Clip line using Cohen-Sutherland algorithm
    Returns: (clipped_x0, clipped_y0, clipped_x1, clipped_y1, accept)
    """
    outcode0 = compute_outcode(x0, y0, x_min, y_min, x_max, y_max)
    outcode1 = compute_outcode(x1, y1, x_min, y_min, x_max, y_max)
    accept = False
    
    while True:
        if outcode0 == 0 and outcode1 == 0:
            # Both points inside
            accept = True
            break
        elif (outcode0 & outcode1) != 0:
            # Both points outside same region
            break
        else:
            # Line needs clipping
            # Pick point outside viewport
            outcode_out = outcode0 if outcode0 != 0 else outcode1
            
            # Find intersection point
            if outcode_out & TOP:
                x = x0 + (x1 - x0) * (y_max - y0) / (y1 - y0)
                y = y_max
            elif outcode_out & BOTTOM:
                x = x0 + (x1 - x0) * (y_min - y0) / (y1 - y0)
                y = y_min
            elif outcode_out & RIGHT:
                y = y0 + (y1 - y0) * (x_max - x0) / (x1 - x0)
                x = x_max
            elif outcode_out & LEFT:
                y = y0 + (y1 - y0) * (x_min - x0) / (x1 - x0)
                x = x_min
            
            # Update point and recompute outcode
            if outcode_out == outcode0:
                x0, y0 = x, y
                outcode0 = compute_outcode(x0, y0, x_min, y_min, x_max, y_max)
            else:
                x1, y1 = x, y
                outcode1 = compute_outcode(x1, y1, x_min, y_min, x_max, y_max)
    
    return (x0, y0, x1, y1, accept)

def draw_line_clipped(fb: Framebuffer, x0: float, y0: float, x1: float, y1: float,
                     color: Tuple[float, float, float],
                     x_min: float = 0, y_min: float = 0, 
                     x_max: float = None, y_max: float = None):
    """Draw line with clipping"""
    if x_max is None:
        x_max = fb.width - 1
    if y_max is None:
        y_max = fb.height - 1
    
    x0_c, y0_c, x1_c, y1_c, accept = clip_line_cohen_sutherland(
        x0, y0, x1, y1, x_min, y_min, x_max, y_max
    )
    
    if accept:
        draw_line_bresenham(fb, int(x0_c), int(y0_c), int(x1_c), int(y1_c), color)

# Test clipping
fb = Framebuffer(600, 600, background_color=(0.1, 0.1, 0.15))

# Define viewport
viewport_x_min, viewport_y_min = 150, 150
viewport_x_max, viewport_y_max = 450, 450

# Draw viewport boundary
draw_line_bresenham(fb, viewport_x_min, viewport_y_min, viewport_x_max, viewport_y_min, (1.0, 1.0, 0.0))
draw_line_bresenham(fb, viewport_x_max, viewport_y_min, viewport_x_max, viewport_y_max, (1.0, 1.0, 0.0))
draw_line_bresenham(fb, viewport_x_max, viewport_y_max, viewport_x_min, viewport_y_max, (1.0, 1.0, 0.0))
draw_line_bresenham(fb, viewport_x_min, viewport_y_max, viewport_x_min, viewport_y_min, (1.0, 1.0, 0.0))

# Draw lines that extend beyond viewport (will be clipped)
test_lines = [
    (100, 300, 500, 300, (1.0, 0.0, 0.0)),  # Horizontal through viewport
    (300, 100, 300, 500, (0.0, 1.0, 0.0)),  # Vertical through viewport
    (100, 100, 500, 500, (0.0, 0.5, 1.0)),  # Diagonal
    (100, 500, 500, 100, (1.0, 0.0, 1.0)),  # Opposite diagonal
    (50, 200, 550, 400, (1.0, 0.5, 0.0)),   # Partially inside
    (0, 0, 200, 100, (0.5, 1.0, 0.5)),      # Partially outside
    (500, 500, 600, 600, (0.7, 0.7, 1.0)),  # Partially outside
]

for x0, y0, x1, y1, color in test_lines:
    draw_line_clipped(fb, x0, y0, x1, y1, color,
                     viewport_x_min, viewport_y_min, viewport_x_max, viewport_y_max)

fb.display("Line Clipping (Cohen-Sutherland)")

---

## 5. 2D Clipping - Implementation

---

## 5. 2D Clipping - Theory

### 5.1 Cohen-Sutherland Line Clipping

**Problem:** Clip lines to a rectangular viewport (window).

Define viewport with boundaries: $x_{\min}, x_{\max}, y_{\min}, y_{\max}$

**Region codes** (4-bit binary):
- Bit 0 (1): point is above viewport ($y > y_{\max}$)
- Bit 1 (2): point is below viewport ($y < y_{\min}$)
- Bit 2 (4): point is right of viewport ($x > x_{\max}$)
- Bit 3 (8): point is left of viewport ($x < x_{\min}$)

**Algorithm:**
1. Compute region codes for both endpoints
2. **Both codes = 0000:** Line completely inside → accept
3. **Bitwise AND ≠ 0:** Line completely outside → reject
4. **Otherwise:** Line partially inside → clip and repeat

**Clipping:** Intersect line with viewport boundary and update endpoint.

### 5.2 Sutherland-Hodgman Polygon Clipping

**Problem:** Clip a polygon against a rectangular viewport.

**Algorithm:** Clip polygon against each edge of viewport sequentially:

For each edge of viewport:
1. Process each polygon edge
2. If edge crosses viewport boundary:
   - Compute intersection point
   - Add appropriate vertices to output polygon

**Output:** New polygon (possibly with more vertices)

### 5.3 Line-Rectangle Intersection

For horizontal/vertical viewport edges, intersection is simple:

**Vertical edge at $x = x_c$:**
$$y_{\text{intersect}} = y_0 + (y_1 - y_0) \frac{x_c - x_0}{x_1 - x_0}$$

**Horizontal edge at $y = y_c$:**
$$x_{\text{intersect}} = x_0 + (x_1 - x_0) \frac{y_c - y_0}{y_1 - y_0}$$

In [ ]:
def edge_function(ax: int, ay: int, bx: int, by: int, px: int, py: int) -> float:
    """Edge function: returns positive if p is to the left of edge a->b"""
    return (px - ax) * (by - ay) - (py - ay) * (bx - ax)

def barycentric_coords(x: int, y: int, 
                       x0: int, y0: int, x1: int, y1: int, x2: int, y2: int) -> Tuple[float, float, float]:
    """Compute barycentric coordinates for point (x,y) in triangle"""
    denom = edge_function(x0, y0, x1, y1, x2, y2)
    
    if abs(denom) < 1e-6:  # Degenerate triangle
        return (-1, -1, -1)
    
    alpha = edge_function(x1, y1, x2, y2, x, y) / denom
    beta = edge_function(x2, y2, x0, y0, x, y) / denom
    gamma = 1.0 - alpha - beta
    
    return (alpha, beta, gamma)

def draw_triangle_wireframe(fb: Framebuffer, 
                            x0: int, y0: int, 
                            x1: int, y1: int, 
                            x2: int, y2: int, 
                            color: Tuple[float, float, float]):
    """Draw triangle outline"""
    draw_line_bresenham(fb, x0, y0, x1, y1, color)
    draw_line_bresenham(fb, x1, y1, x2, y2, color)
    draw_line_bresenham(fb, x2, y2, x0, y0, color)

def draw_triangle_filled(fb: Framebuffer, 
                        x0: int, y0: int, 
                        x1: int, y1: int, 
                        x2: int, y2: int, 
                        color: Tuple[float, float, float]):
    """Draw filled triangle using barycentric coordinates"""
    # Compute bounding box
    min_x = max(0, min(x0, x1, x2))
    max_x = min(fb.width - 1, max(x0, x1, x2))
    min_y = max(0, min(y0, y1, y2))
    max_y = min(fb.height - 1, max(y0, y1, y2))
    
    # Iterate over bounding box
    for y in range(min_y, max_y + 1):
        for x in range(min_x, max_x + 1):
            alpha, beta, gamma = barycentric_coords(x, y, x0, y0, x1, y1, x2, y2)
            
            # Check if point is inside triangle
            if alpha >= 0 and beta >= 0 and gamma >= 0:
                fb.set_pixel(x, y, color)

def draw_triangle_interpolated(fb: Framebuffer, 
                               x0: int, y0: int, c0: Tuple[float, float, float],
                               x1: int, y1: int, c1: Tuple[float, float, float],
                               x2: int, y2: int, c2: Tuple[float, float, float]):
    """Draw triangle with interpolated vertex colors"""
    # Compute bounding box
    min_x = max(0, min(x0, x1, x2))
    max_x = min(fb.width - 1, max(x0, x1, x2))
    min_y = max(0, min(y0, y1, y2))
    max_y = min(fb.height - 1, max(y0, y1, y2))
    
    # Iterate over bounding box
    for y in range(min_y, max_y + 1):
        for x in range(min_x, max_x + 1):
            alpha, beta, gamma = barycentric_coords(x, y, x0, y0, x1, y1, x2, y2)
            
            # Check if point is inside triangle
            if alpha >= 0 and beta >= 0 and gamma >= 0:
                # Interpolate color using barycentric coordinates
                r = alpha * c0[0] + beta * c1[0] + gamma * c2[0]
                g = alpha * c0[1] + beta * c1[1] + gamma * c2[1]
                b = alpha * c0[2] + beta * c1[2] + gamma * c2[2]
                fb.set_pixel(x, y, (r, g, b))

# Test triangle rasterization
fb = Framebuffer(600, 600, background_color=(0.05, 0.05, 0.1))

# Draw wireframe triangles
draw_triangle_wireframe(fb, 100, 100, 200, 150, 150, 200, (1.0, 1.0, 1.0))

# Draw filled triangles
draw_triangle_filled(fb, 300, 100, 500, 150, 400, 250, (1.0, 0.3, 0.3))
draw_triangle_filled(fb, 100, 300, 200, 500, 50, 450, (0.3, 1.0, 0.3))

# Draw triangle with color interpolation
draw_triangle_interpolated(fb, 
                           300, 300, (1.0, 0.0, 0.0),  # Red vertex
                           550, 350, (0.0, 1.0, 0.0),  # Green vertex
                           425, 550, (0.0, 0.0, 1.0))  # Blue vertex

# Draw a more complex shape using multiple triangles
center_x, center_y = 450, 450
radius = 80
n_triangles = 8

for i in range(n_triangles):
    angle1 = 2 * math.pi * i / n_triangles
    angle2 = 2 * math.pi * (i + 1) / n_triangles
    
    x1 = center_x + int(radius * math.cos(angle1))
    y1 = center_y + int(radius * math.sin(angle1))
    x2 = center_x + int(radius * math.cos(angle2))
    y2 = center_y + int(radius * math.sin(angle2))
    
    # Alternating colors
    if i % 2 == 0:
        color = (1.0, 0.7, 0.2)
    else:
        color = (0.2, 0.7, 1.0)
    
    draw_triangle_filled(fb, center_x, center_y, x1, y1, x2, y2, color)

fb.display("Triangle Rasterization")

---

## 4. Triangle Rasterization - Implementation

---

## 4. Triangle Rasterization - Theory

### 4.1 Triangle Filling

A **triangle** is defined by three vertices: $\mathbf{v}_0 = (x_0, y_0)$, $\mathbf{v}_1 = (x_1, y_1)$, $\mathbf{v}_2 = (x_2, y_2)$

**Goal:** Fill all pixels inside the triangle.

### 4.2 Barycentric Coordinates

Any point $\mathbf{p}$ inside (or on) a triangle can be expressed as:

$$\mathbf{p} = \alpha \mathbf{v}_0 + \beta \mathbf{v}_1 + \gamma \mathbf{v}_2$$

where $\alpha + \beta + \gamma = 1$ and $\alpha, \beta, \gamma \geq 0$

**Computing barycentric coordinates:**

$$\alpha = \frac{\text{Area}(\mathbf{p}, \mathbf{v}_1, \mathbf{v}_2)}{\text{Area}(\mathbf{v}_0, \mathbf{v}_1, \mathbf{v}_2)}$$
$$\beta = \frac{\text{Area}(\mathbf{v}_0, \mathbf{p}, \mathbf{v}_2)}{\text{Area}(\mathbf{v}_0, \mathbf{v}_1, \mathbf{v}_2)}$$
$$\gamma = 1 - \alpha - \beta$$

**Area using cross product:**
$$\text{Area}(\mathbf{a}, \mathbf{b}, \mathbf{c}) = \frac{1}{2}|(b_x - a_x)(c_y - a_y) - (c_x - a_x)(b_y - a_y)|$$

**Point is inside triangle if:** $\alpha \geq 0$ AND $\beta \geq 0$ AND $\gamma \geq 0$

### 4.3 Edge Function Method

Define **edge function** for edge from $\mathbf{v}_0$ to $\mathbf{v}_1$:

$$E(\mathbf{p}) = (p_x - v_{0x})(v_{1y} - v_{0y}) - (p_y - v_{0y})(v_{1x} - v_{0x})$$

**Interpretation:**
- $E(\mathbf{p}) > 0$: point is to the left of edge
- $E(\mathbf{p}) = 0$: point is on edge
- $E(\mathbf{p}) < 0$: point is to the right of edge

**Triangle test:** Point inside if all three edge tests have same sign.

### 4.4 Scan-Line Algorithm

Alternative approach:

1. Sort vertices by $y$ coordinate: $y_0 \leq y_1 \leq y_2$
2. For each scan-line $y$ from $y_0$ to $y_2$:
   - Compute intersection with triangle edges
   - Fill pixels between left and right intersections

**Advantage:** Process pixels in scanline order (cache-friendly)

In [ ]:
def draw_circle_midpoint(fb: Framebuffer, xc: int, yc: int, radius: int, 
                         color: Tuple[float, float, float]):
    """Draw circle using Midpoint Circle Algorithm"""
    x = 0
    y = radius
    d = 1 - radius
    
    def plot_circle_points(xc, yc, x, y):
        """Plot 8 symmetric points"""
        fb.set_pixel(xc + x, yc + y, color)
        fb.set_pixel(xc - x, yc + y, color)
        fb.set_pixel(xc + x, yc - y, color)
        fb.set_pixel(xc - x, yc - y, color)
        fb.set_pixel(xc + y, yc + x, color)
        fb.set_pixel(xc - y, yc + x, color)
        fb.set_pixel(xc + y, yc - x, color)
        fb.set_pixel(xc - y, yc - x, color)
    
    plot_circle_points(xc, yc, x, y)
    
    while x < y:
        if d < 0:
            d += 2 * x + 3
        else:
            d += 2 * (x - y) + 5
            y -= 1
        x += 1
        plot_circle_points(xc, yc, x, y)

def draw_circle_filled(fb: Framebuffer, xc: int, yc: int, radius: int, 
                       color: Tuple[float, float, float]):
    """Draw filled circle using scan-line method"""
    for y in range(-radius, radius + 1):
        # For each horizontal line, compute x extent
        x_extent = int(math.sqrt(radius * radius - y * y))
        for x in range(-x_extent, x_extent + 1):
            fb.set_pixel(xc + x, yc + y, color)

def draw_circle_parametric(fb: Framebuffer, xc: int, yc: int, radius: int, 
                           color: Tuple[float, float, float], segments: int = 100):
    """Draw circle using parametric equations"""
    prev_x, prev_y = None, None
    
    for i in range(segments + 1):
        theta = 2 * math.pi * i / segments
        x = xc + int(radius * math.cos(theta))
        y = yc + int(radius * math.sin(theta))
        
        if prev_x is not None:
            draw_line_bresenham(fb, prev_x, prev_y, x, y, color)
        
        prev_x, prev_y = x, y

# Test circle drawing
fb = Framebuffer(500, 500, background_color=(0.05, 0.05, 0.1))

# Draw concentric circles
for r in range(20, 200, 20):
    # Color gradient based on radius
    intensity = r / 200
    draw_circle_midpoint(fb, 250, 250, r, (intensity, 0.5, 1.0 - intensity))

# Draw some filled circles
draw_circle_filled(fb, 100, 100, 30, (1.0, 0.3, 0.3))
draw_circle_filled(fb, 400, 100, 25, (0.3, 1.0, 0.3))
draw_circle_filled(fb, 100, 400, 35, (0.3, 0.3, 1.0))
draw_circle_filled(fb, 400, 400, 28, (1.0, 1.0, 0.3))

fb.display("Circle Drawing Algorithms")

---

## 3. Circle Drawing - Implementation

---

## 3. Circle Drawing - Theory

### 3.1 Midpoint Circle Algorithm

Drawing a circle with center $(x_c, y_c)$ and radius $r$.

**Circle equation:**
$$f(x, y) = (x - x_c)^2 + (y - y_c)^2 - r^2 = 0$$

**Midpoint Circle Algorithm** (Bresenham for circles):

Uses 8-way symmetry: If $(x, y)$ is on the circle, so are:
$$(x, y), (-x, y), (x, -y), (-x, -y), (y, x), (-y, x), (y, -x), (-y, -x)$$

**Algorithm** (for first octant, $0° \leq \theta \leq 45°$):

1. Start at $(0, r)$ relative to center
2. Decision parameter: $d = 1 - r$
3. For each point:
   - If $d < 0$: move East, $d \leftarrow d + 2x + 3$
   - If $d \geq 0$: move Southeast, $d \leftarrow d + 2(x - y) + 5$
4. Plot 8 symmetric points for each computed point

### 3.2 Parametric Circle

Alternative using parametric equations:

$$x = x_c + r\cos\theta$$
$$y = y_c + r\sin\theta, \quad \theta \in [0, 2\pi]$$

**Advantage:** Simple and exact  
**Disadvantage:** Requires trigonometric functions (slower)

In [ ]:
def draw_line_bresenham(fb: Framebuffer, x0: int, y0: int, x1: int, y1: int, 
                         color: Tuple[float, float, float]):
    """Draw line using Bresenham's algorithm"""
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    
    # Determine direction
    sx = 1 if x0 < x1 else -1
    sy = 1 if y0 < y1 else -1
    
    err = dx - dy
    
    x, y = x0, y0
    
    while True:
        fb.set_pixel(x, y, color)
        
        if x == x1 and y == y1:
            break
        
        e2 = 2 * err
        
        if e2 > -dy:
            err -= dy
            x += sx
        
        if e2 < dx:
            err += dx
            y += sy

def draw_line_dda(fb: Framebuffer, x0: int, y0: int, x1: int, y1: int, 
                  color: Tuple[float, float, float]):
    """Draw line using DDA algorithm"""
    dx = x1 - x0
    dy = y1 - y0
    
    steps = max(abs(dx), abs(dy))
    
    if steps == 0:
        fb.set_pixel(x0, y0, color)
        return
    
    x_inc = dx / steps
    y_inc = dy / steps
    
    x, y = float(x0), float(y0)
    
    for _ in range(steps + 1):
        fb.set_pixel(int(round(x)), int(round(y)), color)
        x += x_inc
        y += y_inc

# Test line drawing
fb = Framebuffer(400, 400, background_color=(0.05, 0.05, 0.1))

# Draw several lines radiating from center
center_x, center_y = 200, 200
for angle in range(0, 360, 15):
    rad = math.radians(angle)
    x = center_x + int(150 * math.cos(rad))
    y = center_y + int(150 * math.sin(rad))
    
    # Color gradient based on angle
    r = (math.cos(rad) + 1) / 2
    g = (math.sin(rad) + 1) / 2
    b = (math.cos(rad + math.pi/2) + 1) / 2
    
    draw_line_bresenham(fb, center_x, center_y, x, y, (r, g, b))

# Draw a grid using DDA
for i in range(0, 400, 40):
    draw_line_dda(fb, i, 0, i, 399, (0.2, 0.2, 0.2))
    draw_line_dda(fb, 0, i, 399, i, (0.2, 0.2, 0.2))

fb.display("Line Drawing: Bresenham + DDA")

---

## 2. Line Drawing - Implementation

---

## 2. Line Drawing - Theory

### 2.1 Bresenham's Line Algorithm

Drawing a line from $(x_0, y_0)$ to $(x_1, y_1)$ requires determining which pixels to fill.

**Mathematical line equation:**
$$y = mx + b, \quad m = \frac{y_1 - y_0}{x_1 - x_0}$$

**Problem:** Pixels are discrete, but the line is continuous.

**Bresenham's Algorithm** uses only integer arithmetic:

For a line with $0 \leq m \leq 1$ (gentle slope):

1. Start at $(x_0, y_0)$
2. For each $x$ from $x_0$ to $x_1$:
   - Decide whether to place pixel at $y$ or $y+1$
   - Use error term: $e = 2\Delta y - \Delta x$
   - If $e > 0$: increment $y$ and update $e \leftarrow e + 2(\Delta y - \Delta x)$
   - Otherwise: update $e \leftarrow e + 2\Delta y$

**Key insight:** Uses only integer addition (no multiplication or division) for efficiency.

### 2.2 Digital Differential Analyzer (DDA)

Alternative line drawing algorithm:

$$x_i = x_0 + i \cdot \frac{x_1 - x_0}{steps}$$
$$y_i = y_0 + i \cdot \frac{y_1 - y_0}{steps}$$

where $steps = \max(|x_1 - x_0|, |y_1 - y_0|)$

**Advantage:** Simpler to implement  
**Disadvantage:** Uses floating-point arithmetic (slower than Bresenham)

In [ ]:
class Framebuffer:
    """Framebuffer implementation for rendering"""
    
    def __init__(self, width: int, height: int, background_color: Tuple[float, float, float] = (0.0, 0.0, 0.0)):
        """
        Initialize framebuffer
        
        Args:
            width: Width in pixels
            height: Height in pixels
            background_color: RGB tuple with values in [0, 1]
        """
        self.width = width
        self.height = height
        # Store colors as float32 in range [0, 1] for better precision
        self.buffer = np.zeros((height, width, 3), dtype=np.float32)
        self.clear(background_color)
    
    def clear(self, color: Tuple[float, float, float] = (0.0, 0.0, 0.0)):
        """Clear framebuffer to specified color"""
        self.buffer[:, :] = color
    
    def set_pixel(self, x: int, y: int, color: Tuple[float, float, float]):
        """Set a single pixel (with bounds checking)"""
        if 0 <= x < self.width and 0 <= y < self.height:
            self.buffer[y, x] = color
    
    def get_pixel(self, x: int, y: int) -> Tuple[float, float, float]:
        """Get pixel color at (x, y)"""
        if 0 <= x < self.width and 0 <= y < self.height:
            return tuple(self.buffer[y, x])
        return (0.0, 0.0, 0.0)
    
    def blend_pixel(self, x: int, y: int, color: Tuple[float, float, float], alpha: float):
        """Blend color with existing pixel using alpha"""
        if 0 <= x < self.width and 0 <= y < self.height:
            existing = self.buffer[y, x]
            blended = alpha * np.array(color) + (1 - alpha) * existing
            self.buffer[y, x] = np.clip(blended, 0.0, 1.0)
    
    def display(self, title: str = "Framebuffer"):
        """Display the framebuffer using matplotlib"""
        plt.figure(figsize=(10, 10 * self.height / self.width))
        plt.imshow(self.buffer, origin='lower')  # origin='lower' puts (0,0) at bottom-left
        plt.title(title)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    def save(self, filename: str):
        """Save framebuffer to image file"""
        # Convert to uint8 for saving
        img = (np.clip(self.buffer, 0, 1) * 255).astype(np.uint8)
        plt.imsave(filename, img, origin='lower')

# Test framebuffer
fb = Framebuffer(400, 300, background_color=(0.1, 0.1, 0.2))

# Draw some pixels
for i in range(50):
    x = 200 + int(30 * math.cos(i * 0.3))
    y = 150 + int(30 * math.sin(i * 0.3))
    fb.set_pixel(x, y, (1.0, 0.5, 0.0))

print(f"Created framebuffer: {fb.width}x{fb.height}")
fb.display("Framebuffer Test")

---

## 1. Framebuffer - Implementation

---

## Setup and Imports

In [ ]:
# Your implementation starts here
